In [ ]:
!pip install --upgrade arabic-reshaper
!pip install python-bidi
!pip install qalsadi

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 256.6/256.6 KB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.6/332.6 KB 17.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 360.5/360.5 KB 29.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.4/126.4 KB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 251.5/251.5 KB 20.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.5/14.5 MB 19.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.1/65.1 KB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.0/139.0 KB 12.7 MB/s eta 0:00:00
  Created wheel

In [ ]:
import re
import nltk
nltk.download('popular', quiet=True)
from nltk.tokenize import RegexpTokenizer
import numpy as np
import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20,10)
from collections import Counter
import arabic_reshaper
from bidi.algorithm import get_display
import qalsadi.lemmatizer
from wordcloud import WordCloud
import pandas as pd

In [ ]:
train = pd.read_csv('/content/OSACT2022-sharedTask-train.txt', sep='\t', header=None)
train.columns = ['drop1', 'text', 'drop2', 'label', 'drop3', 'drop4']
train.drop(['drop1', 'drop2', 'drop3', 'drop4'], axis=1, inplace=True)
print(train['label'].value_counts())
print('')
train['label'].replace('HS1', 'HS', inplace=True)
train['label'].replace('HS2', 'HS', inplace=True)
train['label'].replace('HS3', 'HS', inplace=True)
train['label'].replace('HS4', 'HS', inplace=True)
train['label'].replace('HS5', 'HS', inplace=True)
train['label'].replace('HS6', 'HS', inplace=True)
print(train['label'].value_counts())
train

NOT_HS    7634
HS6        437
HS1        251
HS3        139
HS5         69
HS2         27
Name: label, dtype: int64

NOT_HS    7634
HS         923
Name: label, dtype: int64


,text,label
0,@USER ردينا ع التطنز 😏👊🏻,NOT_HS
1,وصارت فطاير البقالات غذاء صحي 👎🏻 URL,NOT_HS
2,@USER روحي لبريده تلقين اشباه كثير بس ماحد زيك...,NOT_HS
3,@USER مش باين حاجه خالص 😣<LF>مش عارف بقى 😔,NOT_HS
4,#اليوم_الاثنين<LF><LF>👏 يقولك :%90 من المسلمي...,NOT_HS
...,...,...
8552,@USER الله يلعنهم 🤢,NOT_HS
8553,واحد سال زوجته بعد كم سنة زواج:<LF>- حبيبتي كم...,NOT_HS
8554,@USER يالله روح زي الشاطر واحذف الشو الي سويته...,NOT_HS
8555,لـمـا الـكـلاب تـهـوهـو عـلـيـك🐶🐶 وأنـت_اللـى_...,NOT_HS


In [ ]:
with open('/content/OSACT2022-sharedTask-test-taskB-gold-labels.txt') as f:
  taskB = f.readlines()
print(len(taskB))
print(dict((x,taskB.count(x)) for x in set(taskB)))
print('')

with open('/content/OSACT2022-sharedTask-test-taskC-gold-labels.txt') as f:
  taskC = f.readlines()
print(len(taskC))
print(dict((x,taskC.count(x)) for x in set(taskC)))

2541
{'HS\n': 271, 'NOT_HS\n': 2270}

2541
{'HS4\n': 2, 'HS1\n': 78, 'HS2\n': 7, 'NOT_HS\n': 2270, 'HS3\n': 32, 'HS5\n': 19, 'HS6\n': 133}


In [ ]:
test = pd.read_csv('/content/OSACT2022-sharedTask-test-tweets.txt', sep='\t', header=None)
test.columns = ['drop', 'text']
test.drop('drop', axis=1, inplace=True)
test['label'] = taskB
test['label'].replace('NOT_HS\n', 'NOT_HS', inplace=True)
test['label'].replace('HS\n', 'HS', inplace=True)
print(test['label'].value_counts())
test

NOT_HS    2270
HS         271
Name: label, dtype: int64


,text,label
0,@USER هتهزر معايا ولا ايه 😡😡😡😡,NOT_HS
1,مشفتش العرض بتاعهم لا مش مهتمة لا😩🐸😂 URL,NOT_HS
2,RT @USER عندما تكون لوحدك تحس انك لحالك صح 😊<L...,NOT_HS
3,RT @USER ماشاء الله الرجال باين عليه محترم <LF...,NOT_HS
4,@USER شسالفة احد يفهمني 😤,NOT_HS
...,...,...
2536,قله حيا وين اهلهم ذولي الله لايبلانا لهالدرجه ...,NOT_HS
2537,RT @USER ثم الطحلبه🐸🐸🐸 URL,NOT_HS
2538,يا وجه الله 😷 من اليوم ورايح شاورما انسى 🔪 URL,NOT_HS
2539,@USER متخلف حتى الحلال حرمتوه 😷,NOT_HS


In [ ]:
with open('/content/OSACT2022-sharedTask-train.txt') as f:
  documents = f.readlines()

In [ ]:
len(documents)

8887

In [ ]:
doc_test = 'مرحبا @hello3 #hello3 999 3 3.9$ , . ! كيفك في على 😋😋😋 نحن ذاهبون إلى أمريكا http://t.co/H0txdjv3Kn'
print('Before:', doc_test)

def TextProcessing(document):
  # Remove emojis
  emoji_pattern = re.compile("["
          u"\U0001F600-\U0001F64F"
          u"\U0001F300-\U0001F5FF"
          u"\U0001F680-\U0001F6FF"
          u"\U0001F1E0-\U0001F1FF"
          u"\U00002702-\U000027B0"
          u"\U000024C2-\U0001F251"
                            "]+", flags=re.UNICODE)
  document = emoji_pattern.sub(r'', document)

  # Remove URLs
  document = re.sub('(https|http).+', '', document)

  # Remove mentions and hashtags
  document = re.sub('[@#]\w+', '', document)

  # Remove unwanted characters
  document = re.sub('[^\s|0-9|ءؤئإآأا-ي|\.\$]', '', document)

  # Replacing the multiple spaces between words with 1 space
  document = re.sub(' +', ' ', document)

  # Tokenization
  regx=RegexpTokenizer('[0-9|ءؤئإآأا-ي|\.\$]+')
  tokens = regx.tokenize(document)

  # Remove stop words
  document_final = []
  arb_stopwords = set(nltk.corpus.stopwords.words("arabic"))
  for word in tokens:
    if word not in arb_stopwords:
      document_final.append(word)

  return document_final

print('')
print('After:', TextProcessing(doc_test))

Before: مرحبا @hello3 #hello3 999 3 3.9$ , . ! كيفك في على 😋😋😋 نحن ذاهبون إلى أمريكا http://t.co/H0txdjv3Kn

After: ['مرحبا', '999', '3', '3.9$', '.', 'كيفك', 'ذاهبون', 'أمريكا']


In [ ]:
train_tokens = []
for i in range(len(train)):
  train_tokens.append(TextProcessing(train['text'].iloc[i]))

train_final = []
sen = ' '
for document in train_tokens:
  sen = ' '.join(document)
  train_final.append(sen)
len(train_final)

8557

In [ ]:
test_tokens = []
for i in range(len(test)):
  test_tokens.append(TextProcessing(test['text'].iloc[i]))

test_final = []
sen = ' '
for document in test_tokens:
  sen = ' '.join(document)
  test_final.append(sen)
len(test_final)

2541

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
vectorizer = CountVectorizer(max_features=1500, min_df=5, max_df=0.7, stop_words=stopwords.words('arabic'))
X_train = vectorizer.fit_transform(train_final).toarray()

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:409: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['آمين', 'أب', 'أخ', 'أفعل', 'أفعله', 'ؤلاء', 'إل', 'إم', 'ات', 'اتان', 'ارتد', 'ان', 'انفك', 'برح', 'تان', 'تبد', 'تحو', 'تعل', 'حد', 'حم', 'حي', 'خب', 'ذار', 'سيما', 'صه', 'ظل', 'ظن', 'عد', 'قط', 'مر', 'مكان', 'مكانكن', 'نب', 'هات', 'هب', 'واها', 'وراء'] not in stop_words.
  warnings.warn(


In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer
tfidfconverter = TfidfTransformer()
X_train = tfidfconverter.fit_transform(X_train).toarray()

In [ ]:
X_test = vectorizer.transform(test_final)
X_test = tfidfconverter.transform(X_test).toarray()

In [ ]:
y_train, y_test = train['label'], test['label']

In [ ]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=100, random_state=0)
classifier.fit(X_train, y_train) 

RandomForestClassifier(random_state=0)

In [ ]:
y_pred = classifier.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  52  219]
 [  42 2228]]
              precision    recall  f1-score   support

          HS       0.55      0.19      0.28       271
      NOT_HS       0.91      0.98      0.94      2270

    accuracy                           0.90      2541
   macro avg       0.73      0.59      0.61      2541
weighted avg       0.87      0.90      0.87      2541

0.89728453364817


In [ ]:
from sklearn.svm import SVC
classifier2 = SVC(random_state=0)
classifier2.fit(X_train, y_train) 

SVC(random_state=0)

In [ ]:
y_pred = classifier2.predict(X_test)
print(confusion_matrix(y_test,y_pred))
print(classification_report(y_test,y_pred))
print(accuracy_score(y_test, y_pred))

[[  27  244]
 [  14 2256]]
              precision    recall  f1-score   support

          HS       0.66      0.10      0.17       271
      NOT_HS       0.90      0.99      0.95      2270

    accuracy                           0.90      2541
   macro avg       0.78      0.55      0.56      2541
weighted avg       0.88      0.90      0.86      2541

0.898465171192444
